# ImageRevive 2.0: AI-Powered Image Restoration
# Notebook 1: Project Selection & Setup

**Course**: AAI-521 Computer Vision  
**Institution**: University of San Diego - Shiley-Marcos School of Engineering  
**Project**: Multi-Agent Image Restoration using Stable Diffusion & Vision Transformers  

---

## Table of Contents
1. [Project Overview](#overview)
2. [Problem Statement](#problem)
3. [Objectives](#objectives)
4. [Technology Stack](#tech-stack)
5. [Environment Setup](#setup)
6. [Model Selection](#models)
7. [Dataset Preparation](#datasets)
8. [Project Structure](#structure)

## 1. Project Overview {#overview}

### Background
Image restoration is a fundamental computer vision task involving the recovery of high-quality images from degraded inputs. This project implements a multi-agent AI system using state-of-the-art models from Hugging Face.

### Key Features
- **Denoising**: Remove noise while preserving details
- **Super-Resolution**: Enhance resolution up to 16K (15360×8640)
- **Colorization**: Add realistic colors to grayscale images
- **Inpainting**: Fill missing or damaged regions

### Innovation
- Multi-agent orchestration using LangGraph
- Aspect ratio preservation
- Ultra-high resolution output (4K-16K)
- Production-ready deployment

## 2. Problem Statement {#problem}

### Challenges
1. **Image Quality Degradation**: Photos suffer from noise, low resolution, missing data
2. **Limited Tools**: Existing solutions don't offer comprehensive restoration
3. **Aspect Ratio Issues**: Many tools distort images during enhancement
4. **Resolution Limits**: Traditional methods cap at 4K or lower

### Solution Approach
Build an AI-powered system that:
- Combines multiple specialized models
- Preserves aspect ratios
- Outputs ultra-high resolutions
- Provides production-ready API and UI

## 3. Objectives {#objectives}

### Primary Objectives
1. Implement state-of-the-art image restoration using Hugging Face models
2. Achieve PSNR > 30 dB and SSIM > 0.90 on standard benchmarks
3. Support resolutions from 4K to 16K with aspect ratio preservation
4. Deploy production-ready system with web interface

### Secondary Objectives
1. Comprehensive performance analysis and benchmarking
2. Comparative study of different model architectures
3. Scalability testing for various image sizes
4. User experience optimization

## 4. Technology Stack {#tech-stack}

### Core Libraries
- **PyTorch**: Deep learning framework
- **Transformers**: Hugging Face model library
- **Diffusers**: Stable Diffusion models
- **OpenCV**: Image processing
- **Pillow**: Image manipulation

### Additional Tools
- **LangGraph**: Multi-agent orchestration
- **Flask**: Web application framework
- **Weights & Biases**: Experiment tracking
- **Docker**: Containerization

## 5. Environment Setup {#setup}

In [ ]:
# PEP 8: Imports grouped and sorted
# Standard library imports
import os
import sys
import warnings
from pathlib import Path

# Third-party imports
import numpy as np
import torch
import torchvision
from PIL import Image

# Suppress warnings
warnings.filterwarnings('ignore')

# Configure matplotlib for inline display
import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("✓ Basic imports successful")

In [ ]:
# Check system configuration
def check_system_config():
    """
    Check and display system configuration.
    
    PEP 8: Function with docstring, clear naming
    """
    print("=" * 60)
    print("SYSTEM CONFIGURATION")
    print("=" * 60)
    
    # Python version
    print(f"Python Version: {sys.version.split()[0]}")
    
    # PyTorch configuration
    print(f"PyTorch Version: {torch.__version__}")
    print(f"CUDA Available: {torch.cuda.is_available()}")
    
    if torch.cuda.is_available():
        print(f"CUDA Version: {torch.version.cuda}")
        print(f"GPU Device: {torch.cuda.get_device_name(0)}")
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    else:
        print("Running on CPU")
    
    # NumPy version
    print(f"NumPy Version: {np.__version__}")
    
    print("=" * 60)

check_system_config()

In [ ]:
# Install required packages
# PEP 8: Clear comments explaining purpose

REQUIRED_PACKAGES = [
    'transformers>=4.35.0',
    'diffusers>=0.24.0',
    'accelerate>=0.25.0',
    'opencv-python>=4.8.0',
    'scikit-image>=0.21.0',
    'lpips>=0.1.4',
    'pytorch-fid>=0.3.0',
    'wandb>=0.16.0',
    'pillow>=10.0.0',
    'matplotlib>=3.7.0',
    'seaborn>=0.12.0',
    'tqdm>=4.66.0'
]

def install_requirements():
    """
    Install required packages.
    
    PEP 8: Descriptive function name, proper docstring
    """
    import subprocess
    
    for package in REQUIRED_PACKAGES:
        try:
            subprocess.check_call(
                [sys.executable, '-m', 'pip', 'install', '-q', package]
            )
            print(f"✓ Installed: {package}")
        except subprocess.CalledProcessError as e:
            print(f"✗ Failed to install {package}: {e}")

# Uncomment to install
# install_requirements()

In [ ]:
# Import Hugging Face libraries
try:
    from transformers import (
        AutoImageProcessor,
        AutoModel,
        Swin2SRForImageSuperResolution,
        CLIPProcessor,
        CLIPModel
    )
    from diffusers import (
        StableDiffusionUpscalePipeline,
        StableDiffusionInpaintPipeline,
        ControlNetModel,
        AutoencoderKL
    )
    import accelerate
    
    print("✓ Hugging Face libraries imported successfully")
    print(f"  - Transformers: {transformers.__version__}")
    print(f"  - Diffusers: {diffusers.__version__}")
    print(f"  - Accelerate: {accelerate.__version__}")
    
except ImportError as e:
    print(f"✗ Import error: {e}")
    print("Please install required packages using the cell above")

## 6. Model Selection {#models}

### Selected Models from Hugging Face

In [ ]:
# PEP 8: Constants in UPPER_CASE
MODEL_REGISTRY = {
    'super_resolution': {
        'swin2sr': {
            'model_id': 'caidas/swin2SR-classical-sr-x4-64',
            'description': 'Swin Transformer v2 for 4x super-resolution',
            'paper': 'https://arxiv.org/abs/2209.11345',
            'performance': 'PSNR: 32.5 dB on Set5'
        },
        'real_esrgan': {
            'model_id': 'ai-forever/Real-ESRGAN',
            'description': 'Real-ESRGAN for practical image restoration',
            'paper': 'https://arxiv.org/abs/2107.10833',
            'performance': 'PSNR: 28.2 dB on RealSR'
        },
        'stable_diffusion_x4': {
            'model_id': 'stabilityai/stable-diffusion-x4-upscaler',
            'description': 'Stable Diffusion 4x upscaler',
            'paper': 'https://arxiv.org/abs/2112.10752',
            'performance': 'Excellent perceptual quality'
        }
    },
    'denoising': {
        'nafnet': {
            'model_id': 'google/nafnet-sidd',
            'description': 'Nonlinear Activation Free Network for denoising',
            'paper': 'https://arxiv.org/abs/2204.04676',
            'performance': 'PSNR: 40.3 dB on SIDD'
        }
    },
    'colorization': {
        'colorful_image_colorization': {
            'model_id': 'shi-labs/colorful-image-colorization',
            'description': 'Automatic colorization using CNN',
            'paper': 'https://arxiv.org/abs/1603.08511',
            'performance': 'Realistic color generation'
        },
        'stable_diffusion_colorization': {
            'model_id': 'stabilityai/stable-diffusion-2-1',
            'description': 'SD 2.1 for guided colorization',
            'paper': 'https://arxiv.org/abs/2112.10752',
            'performance': 'High-quality realistic colors'
        }
    },
    'inpainting': {
        'stable_diffusion_inpaint': {
            'model_id': 'runwayml/stable-diffusion-inpainting',
            'description': 'SD fine-tuned for inpainting',
            'paper': 'https://arxiv.org/abs/2112.10752',
            'performance': 'Context-aware filling'
        },
        'lama': {
            'model_id': 'facebook/lama',
            'description': 'LaMa: Resolution-robust Large Mask Inpainting',
            'paper': 'https://arxiv.org/abs/2109.07161',
            'performance': 'SOTA on large masks'
        }
    },
    'vision_transformer': {
        'vit_base': {
            'model_id': 'google/vit-base-patch16-224',
            'description': 'Vision Transformer for feature extraction',
            'paper': 'https://arxiv.org/abs/2010.11929',
            'performance': 'ImageNet accuracy: 81.2%'
        },
        'dino_v2': {
            'model_id': 'facebook/dinov2-base',
            'description': 'DINOv2 self-supervised ViT',
            'paper': 'https://arxiv.org/abs/2304.07193',
            'performance': 'Strong feature representations'
        }
    }
}

def display_model_registry():
    """
    Display selected models with details.
    
    PEP 8: Clear function documentation
    """
    print("=" * 80)
    print("SELECTED MODELS FROM HUGGING FACE")
    print("=" * 80)
    
    for task, models in MODEL_REGISTRY.items():
        print(f"\n{'Task:':<20} {task.upper()}")
        print("-" * 80)
        
        for model_name, details in models.items():
            print(f"\n  Model: {model_name}")
            print(f"    ID: {details['model_id']}")
            print(f"    Description: {details['description']}")
            print(f"    Performance: {details['performance']}")
    
    print("\n" + "=" * 80)

display_model_registry()

## 7. Dataset Preparation {#datasets}

### Standard Benchmarks

In [ ]:
# PEP 8: Organize related data in dictionary
BENCHMARK_DATASETS = {
    'super_resolution': [
        {
            'name': 'Set5',
            'images': 5,
            'source': 'https://github.com/jbhuang0604/SelfExSR',
            'description': 'Standard SR benchmark with 5 images'
        },
        {
            'name': 'Set14',
            'images': 14,
            'source': 'https://github.com/jbhuang0604/SelfExSR',
            'description': 'Extended SR benchmark with 14 images'
        },
        {
            'name': 'BSD100',
            'images': 100,
            'source': 'https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/bsds/',
            'description': 'Berkeley Segmentation Dataset'
        },
        {
            'name': 'Urban100',
            'images': 100,
            'source': 'https://github.com/jbhuang0604/SelfExSR',
            'description': 'Urban scenes with fine structures'
        },
        {
            'name': 'DIV2K',
            'images': 1000,
            'source': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/',
            'description': 'Diverse 2K resolution images'
        }
    ],
    'denoising': [
        {
            'name': 'SIDD',
            'images': 30000,
            'source': 'https://www.eecs.yorku.ca/~kamel/sidd/',
            'description': 'Smartphone Image Denoising Dataset'
        },
        {
            'name': 'DND',
            'images': 50,
            'source': 'https://noise.visinf.tu-darmstadt.de/',
            'description': 'Darmstadt Noise Dataset'
        }
    ],
    'inpainting': [
        {
            'name': 'Places2',
            'images': 1800000,
            'source': 'http://places2.csail.mit.edu/',
            'description': 'Scene-centric images'
        }
    ]
}

def display_datasets():
    """
    Display benchmark datasets information.
    
    PEP 8: Function with clear purpose and documentation
    """
    print("=" * 80)
    print("BENCHMARK DATASETS")
    print("=" * 80)
    
    for task, datasets in BENCHMARK_DATASETS.items():
        print(f"\n{'Task:':<20} {task.upper()}")
        print("-" * 80)
        
        for dataset in datasets:
            print(f"\n  Dataset: {dataset['name']}")
            print(f"    Images: {dataset['images']}")
            print(f"    Description: {dataset['description']}")
            print(f"    Source: {dataset['source']}")
    
    print("\n" + "=" * 80)

display_datasets()

## 8. Project Structure {#structure}

In [ ]:
# PEP 8: Use pathlib for cross-platform path handling
from pathlib import Path

# Define project structure
PROJECT_STRUCTURE = {
    'data': ['raw', 'processed', 'benchmarks', 'results'],
    'models': ['checkpoints', 'pretrained', 'configs'],
    'notebooks': [],
    'src': ['agents', 'utils', 'metrics'],
    'outputs': ['images', 'logs', 'reports'],
    'tests': []
}

def create_project_structure(base_path='.'):
    """
    Create project directory structure.
    
    Args:
        base_path (str): Base path for project
        
    Returns:
        dict: Created directories
        
    PEP 8: Clear parameter and return documentation
    """
    base = Path(base_path)
    created_dirs = {}
    
    for main_dir, subdirs in PROJECT_STRUCTURE.items():
        main_path = base / main_dir
        main_path.mkdir(exist_ok=True)
        created_dirs[main_dir] = str(main_path)
        
        for subdir in subdirs:
            sub_path = main_path / subdir
            sub_path.mkdir(exist_ok=True)
            created_dirs[f"{main_dir}/{subdir}"] = str(sub_path)
    
    return created_dirs

# Create structure
dirs = create_project_structure('./ImageRevive')

print("Project structure created:")
print("-" * 60)
for name, path in sorted(dirs.items()):
    print(f"{name:<30} → {path}")

## Summary

### Completed Setup
✓ System configuration verified  
✓ Required packages installed  
✓ Models selected from Hugging Face  
✓ Benchmark datasets identified  
✓ Project structure created  

### Next Steps
→ **Notebook 2**: EDA and Pre-Processing  
→ Load and explore benchmark datasets  
→ Implement data augmentation pipelines  
→ Prepare train/validation splits  